In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset, DataLoader


In [ ]:
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')

# Define both dataset paths
drive_t1 = '/content/drive/MyDrive/dataset_t1_new'
drive_t2 = '/content/drive/MyDrive/dataset_t2_new'

# Local paths
local_t1 = '/content/dataset_t1_new'
local_t2 = '/content/dataset_t2_new'

# Copy T1
if not os.path.exists(local_t1):
    shutil.copytree(drive_t1, local_t1)
    print("Copied T1 dataset")
else:
    print("T1 dataset already exists")

# Copy T2
if not os.path.exists(local_t2):
    shutil.copytree(drive_t2, local_t2)
    print("Copied T2 dataset")
else:
    print("T2 dataset already exists")

# Count and print total image files
def count_images_in_folder(folder_path):
    total = 0
    for subdir, _, files in os.walk(folder_path):
        total += len([f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    return total

t1_count = count_images_in_folder(local_t1)
t2_count = count_images_in_folder(local_t2)

print(f"T1 images: {t1_count}")
print(f"T2 images: {t2_count}")


In [ ]:
train_transform = transforms.Compose([
     transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
from torch.utils.data import ConcatDataset, random_split

# Load both datasets with transform
# We will use val_test_transform for validation and test datasets
t1_dataset_train = ImageFolder(root=local_t1, transform=train_transform)
t2_dataset_train = ImageFolder(root=local_t2, transform=train_transform)

t1_dataset_val_test = ImageFolder(root=local_t1, transform=val_test_transform)
t2_dataset_val_test = ImageFolder(root=local_t2, transform=val_test_transform)


# Make sure both datasets are balanced by size (50/50) for the training split
min_len = min(len(t1_dataset_train), len(t2_dataset_train))
t1_balanced_train, _ = random_split(t1_dataset_train, [min_len, len(t1_dataset_train) - min_len])
t2_balanced_train, _ = random_split(t2_dataset_train, [min_len, len(t2_dataset_train) - min_len])

# Merge datasets for the training split
full_dataset_train = ConcatDataset([t1_balanced_train, t2_balanced_train])

# Combine class labels and get indices for the combined training dataset
# The class_to_idx should be consistent across both datasets, so we can use one
class_to_idx = t1_dataset_train.class_to_idx

# Get targets from the balanced subsets for splitting
targets_train = np.array(t1_balanced_train.dataset.targets[:min_len] + t2_balanced_train.dataset.targets[:min_len])

# Create index lists for the combined training dataset based on their original indices
# These indices refer to the positions within the full_dataset_train
false_idx = np.where(targets_train == class_to_idx['false'])[0]
true_idx  = np.where(targets_train == class_to_idx['true'])[0]
np.random.seed(42)
np.random.shuffle(false_idx)
np.random.shuffle(true_idx)

# Split function
def split(indices):
    total = len(indices)
    train_size = int(0.7 * total)
    val_size = int(0.2 * total)
    train = indices[:train_size]
    val = indices[train_size:train_size + val_size]
    test = indices[train_size + val_size:]
    return train, val, test

# Split by class
false_train, false_val, false_test = split(false_idx)
true_train,  true_val,  true_test  = split(true_idx)

# Merge class splits to get indices for the overall train, val, and test sets from the balanced combined dataset
train_idx = np.concatenate([false_train, true_train])
val_idx   = np.concatenate([false_val, true_val])
test_idx  = np.concatenate([false_test, true_test])

# Create datasets using Subsets of the appropriate full dataset
# The training dataset uses the full_dataset_train (with train transforms)
train_dataset = Subset(full_dataset_train, train_idx)

# Create a full dataset with val/test transforms for validation and testing
full_dataset_val_test = ConcatDataset([t1_dataset_val_test, t2_dataset_val_test])

# The validation and test datasets use the full_dataset_val_test (with val_test transforms) and the indices derived from the balanced split
# This is the key change: val_dataset and test_dataset should be subsets of the concatenated dataset,
# not just one of the original ImageFolders.
val_dataset   = Subset(full_dataset_val_test, val_idx)
test_dataset  = Subset(full_dataset_val_test, test_idx)


# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Shuffle should be False for validation and test loaders
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Verify the size of the datasets and loaders
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

print(f"Train loader batches: {len(train_loader)}")
print(f"Validation loader batches: {len(val_loader)}")
print(f"Test loader batches: {len(test_loader)}")

In [ ]:
# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained DenseNet121
model = models.densenet121(pretrained=True)

# Freeze all feature layers
for param in model.features.parameters():
    param.requires_grad = False

# Unfreeze only the last dense block and final normalization layer
for name, param in model.features.named_parameters():
    if 'denseblock4' in name or 'norm5' in name:
        param.requires_grad = True

# Replace classifier with a custom head for binary classification
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(512, 2))

# Move model to device
model.to(device)

# Define loss with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Adam optimizer with low LR and weight decay
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)

# Scheduler to reduce LR if validation loss plateaus
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

In [ ]:
# Train the model with early stopping and learning rate scheduling.
# Tracks the train and validation loss and accuracy each epoch.
# Stops training if validation loss does not improve after 'patience_limit' epochs.

num_epochs = 75
patience_limit = 5
train_losses, val_losses = [], []
train_accs, val_accs = [], []
best_val_loss = float('inf')
patience_counter = 0
for epoch in range(num_epochs):
    print(f"\n Epoch {epoch+1}/{num_epochs}")
    model.train()
    correct = total = 0
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    model.eval()
    val_loss = val_correct = val_total = 0.0
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_epoch_loss = val_loss / val_total
    val_epoch_acc = val_correct / val_total
    val_losses.append(val_epoch_loss)
    val_accs.append(val_epoch_acc)
    scheduler.step(val_epoch_loss)
    print(f"Val Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_acc:.4f}")
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience_limit:
            print("Early stopping")
            break

In [ ]:
# Final evaluation on test set
model.eval()
test_loss, test_correct, test_total = 0.0, 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

test_loss /= test_total
test_acc = test_correct / test_total
print(f"\n Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Collect all true and predicted labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
tn, fp, fn, tp = cm.ravel()

# Compute metrics
accuracy  = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall    = tp / (tp + fn) if (tp + fn) > 0 else 0
f1        = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Print results
print(f"\nConfusion Matrix:\n{cm}")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Endometriosis", "Endometriosis"])
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix on Test Set")
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from google.colab import files
from PIL import Image
import io
import torch.nn.functional as F
import matplotlib.pyplot as plt # Import matplotlib

# Prompt user to upload an image
print("Upload an image to classify:")
uploaded = files.upload()

# Check for uploaded image
if uploaded:
    for fname in uploaded:
        # Load and preprocess image
        image = Image.open(io.BytesIO(uploaded[fname])).convert('RGB')
        plt.imshow(image)
        plt.title("Uploaded Image")
        plt.axis('off')
        plt.show()

        # Apply test transform (resize, normalize)
        input_tensor = val_test_transform(image).unsqueeze(0).to(device)

        # Predict
        model.eval()
        with torch.no_grad():
            output = model(input_tensor)
            probs = F.softmax(output, dim=1)
            _, prediction = torch.max(probs, 1)
            confidence = probs[0][prediction].item()

        # Map index to label
        # Use class_to_idx from one of the original ImageFolder datasets
        # before concatenation, as ConcatDataset does not have this attribute.
        idx_to_class = {v: k for k, v in t1_dataset_val_test.class_to_idx.items()}
        predicted_label = idx_to_class[prediction.item()]

        print(f"Prediction: {predicted_label.upper()}  (Confidence: {confidence:.2%})")